ETL

In [ ]:
import pandas as pd #ler arquivos

import numpy as np
import pandas as pd

In [ ]:
def read_files(path, name_file, year_date, type_file):
  _file = f'{path}{name_file}{year_date}.{type_file}'

  colspecs = [(2, 10), #data pregao
              (10,12), #codBDI
              (12,24),
              (27,39),
              (56,69),
              (69,82),
              (82,95),
              (108,121),
              (152,170),
              (170,188)
  ]

  names = ['data_pregao', 'codbdi', 'sigla_acao', 'nome_acao', 'preco_abertura',
         'preco_maximo', 'preco_minimo', 'preco_fechamento', 'qtd_negocios', 'volume_negocio']    
  print(_file)
  df = pd.read_fwf(_file, colspecs = colspecs, names = names, skiprows = 1, ) #caminho do txt, .., pula a head do arquivo
  
  return df


In [ ]:
#transformar campo em data, float, e eliminar quem nao for do lote padrao (2)

#Filtrar Acoes
def filter_stocks(df):
  df = df [df['codbdi'] == 2] #filtar todos lotes que sejam padrao 2
  df = df.drop(['codbdi'], 1) #agr excluo a coluna ja que nao preciso dela
  df = df.drop(['nome_acao'], 1) #confirmar esse 1
  df = df[df['sigla_acao'] == 'PETR4'] #filtro todas acoes da petrobras
  return df

In [ ]:
#Ajuste campo de data
def parse_date(df):
  df['data_pregao'] = pd.to_datetime(df['data_pregao'], format = '%Y-%m-%d')

  return df

In [ ]:
#Ajuste campos numericos, quero 2 casas decimais e passo o tipo float
def parse_values(df):
  df['preco_abertura'] = (df['preco_abertura'] / 100).astype(float)
  df['preco_maximo'] = (df['preco_maximo'] / 100).astype(float)
  df['preco_minimo'] = (df['preco_minimo'] / 100).astype(float)
  df['preco_fechamento'] = (df['preco_fechamento'] / 100).astype(float)
  
  return df

#df.dtypes

In [ ]:
#Juntando os arquivos

def concat_files(path, name_file, year_date, type_file, final_file):
  
  for i, y in enumerate(year_date):
    df = read_files(path, name_file, y, type_file)
    df = filter_stocks(df)
    df = parse_date(df)
    df = parse_values(df)

    if i==0:
      df_final = df
    else:
      df_final = pd.concat([df_final, df])
    
    df_final.to_csv(f'{path}//{final_file}', index=False) #index=false nao coloca um indice no meu arquivo, usa o indice do pregao

In [ ]:
#Executando programa de ETL

year_date = ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011','2012', 
             '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
path=f'C:\\Users\\Gabriel\\Desktop\\Projeto_LSTM\\LSTM\\ETL\\Series_historicas\\'
name_file='COTAHIST_A' #se num futuro mudar o nome do arquivo
type_file='TXT'
final_file='all_bovespa.csv'

concat_files(path, name_file, year_date, type_file, final_file)

,data_pregao,sigla_acao,preco_abertura,preco_maximo,preco_minimo,preco_fechamento,qtd_negocios,volume_negocio
0,2002-01-01,PETR4,6.39,6.39,6.39,6.39,3.186628,0.0
1,2002-01-02,PETR4,6.45,6.49,6.39,6.41,3.192857,23920000.0
2,2002-01-03,PETR4,6.49,6.50,6.42,6.45,3.214662,39043200.0
3,2002-01-04,PETR4,6.38,6.45,6.32,6.41,3.194726,21500800.0
4,2002-01-07,PETR4,6.38,6.60,6.38,6.55,3.264501,40406400.0
...,...,...,...,...,...,...,...,...
5093,2022-05-06,PETR4,32.69,33.12,31.90,33.06,33.060001,129570200.0
5094,2022-05-09,PETR4,32.70,33.04,32.06,32.16,32.160000,92314500.0
5095,2022-05-10,PETR4,32.36,32.85,32.20,32.44,32.439999,60330600.0
5096,2022-05-11,PETR4,32.78,33.91,32.69,33.57,33.570000,108895400.0
